In [ ]:
import os
import glob
from office365.sharepoint.client_context import ClientContext

def find_latest_file(directory, pattern):
    """
    Searches the specified directory for files matching the given pattern
    and returns the file with the most recent creation time.
    """
    files = glob.glob(os.path.join(directory, pattern))
    if not files:
        raise ValueError(f"No files found matching pattern: {pattern}")
    latest_file = max(files, key=os.path.getctime)
    return latest_file

def upload_file():
    # Get SharePoint configuration from environment variables
    sharepoint_site_url = os.environ['SHAREPOINT_SITE_URL']
    target_folder_relative_url = os.environ['TARGET_FOLDER_RELATIVE_URL']
    
    # Use app-based authentication if SP_CLIENT_ID and SP_CLIENT_SECRET are provided,
    # otherwise fall back to user credentials.
    client_id = os.environ.get('SP_CLIENT_ID')
    client_secret = os.environ.get('SP_CLIENT_SECRET')
    
    if client_id and client_secret:
        from office365.runtime.auth.client_credential import ClientCredential
        credentials = ClientCredential(client_id, client_secret)
        ctx = ClientContext(sharepoint_site_url).with_credentials(credentials)
    else:
        username = os.environ['SP_USER']
        password = os.environ['SP_PASSWORD']
        from office365.runtime.auth.user_credential import UserCredential
        ctx = ClientContext(sharepoint_site_url).with_credentials(UserCredential(username, password))
    
    # Find the latest Excel file in the 'src' folder with the timestamped name
    local_file_path = find_latest_file('src', 'NY_Senate_bills_data_*.xlsx')
    file_name = os.path.basename(local_file_path)
    print(f"Uploading file: {file_name}")
    
    target_folder = ctx.web.get_folder_by_server_relative_url(target_folder_relative_url)
    
    with open(local_file_path, 'rb') as file_content:
        target_folder.upload_file(file_name, file_content.read()).execute_query()
    
    print(f"{file_name} uploaded successfully to SharePoint.")

upload_file()
